In [1]:
import pandas as pd

In [15]:
df = pd.read_csv("cleanData.csv")
df.head()
df = df[df["airlineName_Spirit Airlines"] > 0]
df = df.drop(df.columns[-2:], axis=1)
df = df.rename(columns={"airlineName_American Airlines": "isAA"})
df.head()

,Unnamed: 0,flightDate,travelDuration,isOneElapsedDays,isBasicEconomy,baseFare,seatsRemaining,isOtherEquipment,isFirstCabin,searchDaysBeforeDeparture,departureTimeInMin,isAA
0,2,2022-04-17,186,0.0,0.0,140.0,0,1.0,0.0,1,550,0.0
5,31,2022-04-18,186,0.0,0.0,140.0,0,1.0,0.0,2,550,0.0
10,59,2022-04-19,184,0.0,0.0,100.0,0,0.0,0.0,3,1080,0.0
11,60,2022-04-19,186,0.0,0.0,100.0,0,1.0,0.0,3,550,0.0
16,108,2022-04-20,178,0.0,0.0,100.0,0,0.0,0.0,4,1259,0.0


In [16]:
categorical_vars = [_ for _ in df.columns if _.startswith("is")]
categorical_vars

['isOneElapsedDays',
 'isBasicEconomy',
 'isOtherEquipment',
 'isFirstCabin',
 'isAA']

In [21]:
df_cat = df[categorical_vars].astype(int)
df_cat.head()

,isOneElapsedDays,isBasicEconomy,isOtherEquipment,isFirstCabin,isAA
0,0,0,1,0,0
5,0,0,1,0,0
10,0,0,0,0,0
11,0,0,1,0,0
16,0,0,0,0,0


In [22]:
from itertools import product
import scipy.stats as ss

cat_var_prod = list(product(categorical_vars, categorical_vars, repeat=1))
result = [(i[0], i[1], list(ss.chi2_contingency(pd.crosstab(df_cat[i[0]], df_cat[i[1]])))[1]) for i in cat_var_prod if i[0] != i[1]]

chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 
                                                       'coeff'])
## Using pivot function to convert the above DataFrame into a crosstab
chi_test_output.pivot(index='var1', columns='var2', values='coeff')

var2,isAA,isBasicEconomy,isFirstCabin,isOneElapsedDays,isOtherEquipment
var1,,,,,
isAA,NaN,1.0,1.0,1.00000,1.00000
isBasicEconomy,1.0,NaN,1.0,1.00000,1.00000
isFirstCabin,1.0,1.0,NaN,1.00000,1.00000
isOneElapsedDays,1.0,1.0,1.0,NaN,0.02375
isOtherEquipment,1.0,1.0,1.0,0.02375,NaN


We can drop isOtherEquipment because it is correlated with isOneElapsedDays

In [23]:
df[categorical_vars] = df_cat
df = df.drop('isOtherEquipment', axis=1)

In [24]:
df.head()

,Unnamed: 0,flightDate,travelDuration,isOneElapsedDays,isBasicEconomy,baseFare,seatsRemaining,isFirstCabin,searchDaysBeforeDeparture,departureTimeInMin,isAA
0,2,2022-04-17,186,0,0,140.0,0,0,1,550,0
5,31,2022-04-18,186,0,0,140.0,0,0,2,550,0
10,59,2022-04-19,184,0,0,100.0,0,0,3,1080,0
11,60,2022-04-19,186,0,0,100.0,0,0,3,550,0
16,108,2022-04-20,178,0,0,100.0,0,0,4,1259,0


In [25]:
df.to_csv("cleanData_AA_DL.csv")